In [2]:
import azureml.core
print('SDK version:', azureml.core.VERSION)

SDK version: 1.37.0


In [3]:
from azureml.core import Workspace

ws = Workspace(subscription_id="5aeeb535-21e1-4410-a3d0-539fa06445d7",
               resource_group="SignRecognition",
               workspace_name="SignRecognition")
ws

Workspace.create(name='SignRecognition', subscription_id='5aeeb535-21e1-4410-a3d0-539fa06445d7', resource_group='SignRecognition')

In [16]:
from azureml.core import Model

opset = 11

model = Model.register(workspace=ws,
                       model_name=f'glamorous-wave-{opset}',                  # Name of the registered model in your workspace.
                       model_path=f'../deploy-model/gw-model/model/glamorous-wave-{opset}.onnx',              # Local ONNX model to upload and register as a model.
                      #  model_framework=Model.Framework.ONNX ,      # Framework used to create the model.
                      #  model_framework_version='1.3',              # Version of ONNX used to create the model.
                       description='Onnx Sign Recognition model')

print('Name:', model.name)

Registering model glamorous-wave-11
Name: glamorous-wave-11


In [17]:
from azureml.core import Webservice
from azureml.exceptions import WebserviceException

service_name = f'onnx-sign-recognition-service-{opset}'

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

service = Model.deploy(ws, service_name, [model])
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-12-30 15:32:41+08:00 Creating Container Registry if not exists.
2021-12-30 15:32:41+08:00 Registering the environment.
2021-12-30 15:32:43+08:00 Use the existing image.
2021-12-30 15:32:44+08:00 Submitting deployment to compute.
2021-12-30 15:32:51+08:00 Checking the status of deployment onnx-sign-recognition-service-11..
2021-12-30 15:33:54+08:00 Checking the status of inference endpoint onnx-sign-recognition-service-11.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [20]:
import requests

headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}

if service.auth_enabled:
    headers['Authorization'] = 'Bearer '+ service.get_keys()[0]
elif service.token_auth_enabled:
    headers['Authorization'] = 'Bearer '+ service.get_token()[0]

scoring_uri = service.scoring_uri
print(scoring_uri)
with open('onnx-predict-input.json', 'rb') as data_file:
    response = requests.post(
        scoring_uri, data=data_file, headers=headers)
print(response.status_code)
print(response.elapsed)
print(response.json())

http://7f16a863-1e4a-4b96-b55d-e11c7f9a4fa8.southeastasia.azurecontainer.io/score
502
0:00:00.155233
{'error_code': 500, 'error_message': "ONNX Runtime Status Code: 6. Non-zero status code returned while running Slice node. Name:'hcn/strided_slice' Status Message: /onnxruntime/include/onnxruntime/core/framework/op_kernel.h:90 const T* onnxruntime::OpKernelContext::Input(int) const [with T = onnxruntime::Tensor] Missing Input: input\nStacktrace:\n"}


## Deploy

#### Register model

In [54]:
m_name = "glamorous-wave"
opset = 9
model = Model.register(model_path = f"../models/onnx/{m_name}-{opset}.onnx",
                       model_name=f'{m_name}-{opset}',
                       tags = {"onnx": "demo"},
                       description = "ONNX Sign Recognition HCN",
                       workspace = ws)

Registering model glamorous-wave-9


In [55]:
models = ws.models
for name, m in models.items():
    print("Name:", name,"\tVersion:", m.version, "\tDescription:", m.description, m.tags)

Name: glamorous-wave 	Version: 1 	Description:  {}
Name: glamorous-wave-13 	Version: 1 	Description: Onnx Sign Recognition model {}
Name: glamorous-wave-11 	Version: 1 	Description: Onnx Sign Recognition model {}
Name: glamorous-wave-9 	Version: 2 	Description: ONNX Sign Recognition HCN {'onnx': 'demo'}


In [56]:
models

{'glamorous-wave': Model(workspace=Workspace.create(name='SignRecognition', subscription_id='5aeeb535-21e1-4410-a3d0-539fa06445d7', resource_group='SignRecognition'), name=glamorous-wave, id=glamorous-wave:1, version=1, tags={}, properties={}),
 'glamorous-wave-13': Model(workspace=Workspace.create(name='SignRecognition', subscription_id='5aeeb535-21e1-4410-a3d0-539fa06445d7', resource_group='SignRecognition'), name=glamorous-wave-13, id=glamorous-wave-13:1, version=1, tags={}, properties={}),
 'glamorous-wave-11': Model(workspace=Workspace.create(name='SignRecognition', subscription_id='5aeeb535-21e1-4410-a3d0-539fa06445d7', resource_group='SignRecognition'), name=glamorous-wave-11, id=glamorous-wave-11:1, version=1, tags={}, properties={}),
 'glamorous-wave-9': Model(workspace=Workspace.create(name='SignRecognition', subscription_id='5aeeb535-21e1-4410-a3d0-539fa06445d7', resource_group='SignRecognition'), name=glamorous-wave-9, id=glamorous-wave-9:2, version=2, tags={'onnx': 'demo'}

## Setting up inference config

In [25]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(pip_packages=["numpy", "onnxruntime", "azureml-core", "azureml-defaults"])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [26]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment

myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

In [57]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'demo': 'onnx'}, 
                                               description = 'web service for Sign Recognition ONNX model')

In [58]:
aci_service_name = 'aci-service-sign-recognition'
print("Service", aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

Service aci-service-sign-recognition
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-12-30 22:51:05+08:00 Creating Container Registry if not exists.
2021-12-30 22:51:05+08:00 Registering the environment.
2021-12-30 22:51:06+08:00 Use the existing image.
2021-12-30 22:51:06+08:00 Generating deployment configuration.
2021-12-30 22:51:07+08:00 Submitting deployment to compute.
2021-12-30 22:51:13+08:00 Checking the status of deployment aci-service-sign-recognition..
2021-12-30 22:52:46+08:00 Checking the status of inference endpoint aci-service-sign-recognition.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [59]:
print(aci_service.scoring_uri)

http://a0f6ee0c-5224-40a0-8399-1bdc54e1dd56.southeastasia.azurecontainer.io/score


# Consume